In [1]:
import pandas as pd
import numpy as np

temp_df = pd.read_csv("C:\\Users\\Лев\\Projects\\Diploma\\sbp2.csv", sep=';')
temp_df['time'] = pd.to_datetime(temp_df['time'], dayfirst=True)
temp_df = temp_df.sort_values(by=['time'])
temp = pd.DataFrame(pd.date_range(start=temp_df['time'][55215], end=temp_df['time'][0], freq='3 h'), columns=['time'])
temp = temp.set_index('time').join(temp_df.set_index('time'), how='left')
temp = temp['T'].ffill().to_frame()
temp.index = pd.date_range(start=temp_df['time'][55215], end=temp_df['time'][0], freq='3 h')

from sktime.forecasting.model_selection import temporal_train_test_split

temp_train, temp_test = temporal_train_test_split(temp, test_size=2920)

In [2]:
day_diff = [np.mean([temp_train.iloc[8 * i + j + 1] - temp_train.iloc[8 * i + j] for i in range(1, 6560)]) for j in range(8)]
day_diff

[-0.687116938557707,
 -0.3238450983381614,
 1.2071657264826954,
 1.6168470803476143,
 0.8885043451745691,
 -0.3664735477969202,
 -1.2268638511968288,
 -1.1062966915688366]

In [7]:
np.concatenate(([temp_train.values[-1]], temp_test.values))[:,0].shape

(2921,)

In [29]:
diffs_train = np.diff(temp_train['T'])
diffs_train = np.array([diffs_train[i] - day_diff[i % 8] for i in range(len(diffs_train))])
diffs_test = np.diff(np.concatenate(([temp_train.values[-1]], temp_test.values))[:,0])
diffs_test = np.array([diffs_test[i] - day_diff[i % 8] for i in range(len(diffs_test))])

In [30]:
diffs_test

array([ 0.98711694,  0.0238451 , -1.30716573, ...,  0.76647355,
       -1.07313615,  0.70629669])

In [48]:
X_train = np.array([diffs_train[i:i+13] for i in range(0, 52480, 2)])
Y_train = np.array([diffs_train[i+13] for i in range(0, 52480, 2)])
X_test = np.array([diffs_test[i:i+13] for i in range(0, 2907, 2)])
Y_test = np.array([diffs_test[i+13] for i in range(0, 2907, 2)])

In [32]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(n_estimators = 300,
                             n_jobs=-1)
regr.fit(X_train, Y_train)

RandomForestRegressor(n_estimators=300, n_jobs=-1)

In [22]:
def l2_improved(y_true, y_pred, start_index):
    if len(y_pred) != len(y_true):
        print("Lenghts are not the same!")
        return 0
    hour_vars = [0.770578, 0.727813, 0.729499, 0.885727, 0.982857, 0.999754, 1.000000, 0.883696]
    s = 0
    n = len(y_true)
    for i in range(n):
        s += (y_true[i] - y_pred[i]) ** 2 / hour_vars[(start_index + i) % 8]
    return s / n

In [35]:
regr.predict(X_test)

array([ 0.44384307,  0.86931311, -0.45232718, ..., -0.9869254 ,
       -0.02730396, -0.13373904])

In [34]:
l2_improved(regr.predict(X_test), Y_test, 0)

2.922765927357314

In [57]:
X_test = np.concatenate((np.array([diffs_test[i + 1:i + 13] for i in range(0, 2906, 2)]), np.transpose([regr.predict(X_test)[:-1]])), axis=1)
Y_test = np.array([diffs_test[i + 13] + diffs_test[i + 14] for i in range(0, 2906, 2)])

In [58]:
l2_improved(regr.predict(X_test), Y_test, 0)

7.8852391584946835